In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import time
import odrive
import serial
import pandas as pd

In [ ]:
odrv0 = odrive.find_any()
print(str(odrv0.vbus_voltage))

In [ ]:
# Configure Power Supply

odrv0.config.dc_bus_overvoltage_trip_level = 56 # [V]
odrv0.config.dc_bus_undervoltage_trip_level = 32 # [V]
odrv0.config.dc_max_positive_current = 7.8 # [A]
odrv0.config.dc_max_negative_current = -2 # [A]

# Motor Configuration

odrv0.axis0.config.motor.motor_type = 0 # HIGH_CURRENT
odrv0.axis0.config.motor.torque_constant = 8.27 / 100
odrv0.axis0.config.motor.pole_pairs = 20
odrv0.axis0.config.motor.current_soft_max = 50
odrv0.axis0.config.motor.current_hard_max = 70
odrv0.axis0.config.motor.calibration_current = 10
odrv0.axis0.config.motor.resistance_calib_max_voltage = 2
odrv0.axis0.config.calibration_lockin.current = 10

odrv0.axis0.requested_state = 4 # MOTOR_CALIBRATION

print(f"Phase resistance: {odrv0.axis0.config.motor.phase_resistance} Ohm")
print(f"Phase inductance: {odrv0.axis0.config.motor.phase_inductance} H")

odrv0.save_configuration() # saving the configuration

In [ ]:
# Setting the Limits

odrv0.axis0.controller.config.vel_limit = 25 # [turn/s]
odrv0.axis0.controller.config.vel_limit_tolerance = 1.8 # [turn/s]
odrv0.axis0.config.torque_soft_min = -4 # [Nm]
odrv0.axis0.config.torque_soft_max = 4 # [Nm]

# Encoder Configuration

odrv0.axis0.config.load_encoder = 13 # ONBOARD_ENCODER0
odrv0.axis0.config.commutation_encoder = 13 # ONBOARD_ENCODER0

odrv0.axis0.requested_state = 7 # ENCODER_OFFSET_CALIBRATION

In [ ]:
# Setting up the control loop

odrv0.axis0.requested_state = 8 # CLOSED_LOOP_CONTROL
odrv0.axis0.controller.config.input_mode = 1 # PASSTHROUGH
odrv0.axis0.controller.config.control_mode = 3 # POSITION_CONTROL

# tuned gains
odrv0.axis0.controller.config.pos_gain = 20.0*1.3**4
odrv0.axis0.controller.config.vel_gain = 0.5*(0.16*1.3**9)
odrv0.axis0.controller.config.vel_integrator_gain = 0#0.32

# serial port of ODrive
ser = serial.Serial('COM11', 115200, timeout=0.050)

In [ ]:
def log_signals(startTime, startPos, Time, Pos, Vel, Trq):
    Time.append(time.time() - startTime) # time from start [s]
    Pos.append((odrv0.axis0.pos_vel_mapper.pos_rel - startPos)*2*np.pi) # rel. pos [rad]
    Vel.append(odrv0.axis0.pos_vel_mapper.vel*2*np.pi) # vel. [rad/s]
    Trq.append(odrv0.axis0.motor.torque_estimate) # estim. motor torque [Nm]
    return Time, Pos, Vel, Trq

def deg2turn(signal):
    return signal/360

In [ ]:
endTime = 25

plate = "metal" # some unique name for each surface

# experiment:   0 - lateral stiffnes identification
#               1 - sinusoidal trajectory
#               2 - sinusoidal, increasing amplitude trajectory
experiment = 2

SinAmpl = 60 # [deg] - amplitude of sinusoidal signal
SinFreq = 2 # [rad/s] - frequency of sinusoidal signals

desTime = np.linspace(0, endTime, endTime*8000)
if experiment == 0:
    desPos = deg2turn(np.linspace(0, -10, endTime*8000))
    savefilename = f"{plate}_k_ident.csv"
elif experiment == 1:
    desPos = deg2turn(SinAmpl*np.sin(SinFreq*desTime))
    savefilename = f"{plate}_sine_ampl{SinAmpl}_omega{SinFreq}.csv"
else:
    desPos = deg2turn(np.sin(desTime*SinFreq)*np.linspace(5, 60, endTime*8000))
    savefilename = f"{plate}_sine_incr_ampl_omega{SinFreq}.csv"

plt.plot(desTime, desPos)
plt.show()

In [ ]:
# Experiment

startTime = time.time()
startPos = odrv0.axis0.pos_vel_mapper.pos_rel
currTime = 0

Time = []
Pos = []
Vel = []
Trq = []

sampling_freq = 1/1000
samplings = np.linspace(0, endTime, num=int(endTime/sampling_freq + 1))
samp_count = 0

while currTime < endTime:
    currTime = time.time() - startTime
    posCommand = np.interp(currTime, desTime, desPos) + startPos
    strCommand = f"p 0 {posCommand}\r"
    ser.write(strCommand.encode(encoding="ascii"))
        
    if currTime >= samplings[samp_count]:
        Time, Pos, Vel, Trq = log_signals(startTime, startPos, Time, Pos, Vel, Trq)
        samp_count += 1

In [ ]:
# Saving the measurement

Timen = np.array(Time)
Posn = np.array(Pos)
Veln = np.array(Vel)
Trqn = np.array(Trq)
Data = np.column_stack((Timen, Posn, Veln, Trqn))
df = pd. DataFrame(Data)

df.to_csv(savefilename, index=False, header=False)

plt.figure()
plt.plot(Time, Pos)
plt.show()

plt.figure()
plt.plot(Time, Trq)
plt.show()

In [ ]:
odrv0.axis0.requested_state = 1 # IDLE
ser.close()